In [1]:
from transformers import BertModel, DistilBertModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
import pickle as pkl
import os
from os.path import join as oj
from spacy.lang.en import English
import argparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from datasets import load_from_disk
import sklearn
import warnings
import config

2022-07-07 19:01:06.056586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-07 19:01:06.056633: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/accounts/projects/vision/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


We're going to aggregate embeddings run for a particular order of n-gram here (we'll just add them up).

In [ ]:
for dset in os.listdir(config.data_dir):
    processed_dir = oj(config.data_dir, dset)
    dir_names = [f for f in sorted(os.listdir(processed_dir))
                 if not '-all' in f
                ]

    for i in tqdm(range(len(dir_names))):
        s = dir_names[i]
        start = s.index('=') + 1
        end = s.index('_')
        num = int(s[start: end])
#         if num > 1:
        print('Trying', s)
        s_new = s + '-all'
        """
        if os.path.exists(oj(processed_dir, s_new)):
            print('\tdone already!')
            continue
        """

        pre = s[:start]
        end = s[end:]
        all_exist = True
        num_missing = []
        for num_small in range(1, num):
            fname_small = pre + str(num_small) + end
            all_exist = all_exist and (fname_small in dir_names)
            if not fname_small in dir_names:
                num_missing.append(num_small)

        if not all_exist:
            print('\tmissing small_ngrams: ', num_missing)
            continue

        # load dset
        reloaded_dataset = load_from_disk(oj(processed_dir, s))
        X_train = np.array(reloaded_dataset['train']['embs']).squeeze()
        X_val = np.array(reloaded_dataset['validation']['embs']).squeeze()

        for num_small in range(1, num + 1):
            fname_small = pre + str(num_small) + end            
            reloaded_dataset = load_from_disk(oj(processed_dir, fname_small))
            X_train_small = np.array(reloaded_dataset['train']['embs']).squeeze()
            X_val_small = np.array(reloaded_dataset['validation']['embs']).squeeze()

            X_train += X_train_small
            X_val += X_val_small

        os.makedirs(oj(processed_dir, s_new), exist_ok=True)
        mu = X_train.mean(axis=0)
        sigma = X_train.std(axis=0)
        r = {
            'X_train': X_train,
            'X_val': X_val,
            'mean': mu,
            'sigma': sigma,
        }
        pkl.dump(r, open(oj(processed_dir, s_new, 'data.pkl'), 'wb'))
        print('\tsuccess!')